# Practical demo of bootstrap process for linear polymer structure building and MD setup from scratch

In [ ]:
# Warning suppression and logging
import warnings 
warnings.catch_warnings(record=True)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

import logging
from polymerist.genutils.logutils.IOHandlers import LOG_FORMATTER

logging.basicConfig(
    level=logging.INFO,
    format =LOG_FORMATTER._fmt,
    datefmt=LOG_FORMATTER.datefmt,
    force=True
)
LOGGER = logging.getLogger(__name__)
from IPython.display import clear_output

# Chemistry tools
from rdkit import Chem
from rdkit.Chem.AllChem import EmbedMolecule

from polymerist.rdutils import rdkdraw # configure molecule drawing
rdkdraw.set_rdkdraw_size(400, 3/2)
rdkdraw.disable_substruct_highlights()

# define output directory
from pathlib import Path

outdir = Path('polymer_demos')
outdir.mkdir(exist_ok=True)

# 0) Predefining some monomer examples for you to play with

In [2]:
from dataclasses import dataclass, field
from enum import Enum, auto

class PortMethod(Enum):
    '''For specifying how ports should be added to a complete RDMol'''
    MAP_NUMBERS   = auto()
    BOND_CLEAVAGE = auto()
    REACTION      = auto() # not included here : provide an MDL rxn template

@dataclass
class MonomerExample:
    '''For encapsulating info about a base monomer unit'''
    molname : str
    smiles  : str
    bond_map_nums  : tuple[int, ...]
    isotope_map    : dict[int, int] = field(default_factory=dict)
    assign_port_by : PortMethod = field(default=PortMethod.BOND_CLEAVAGE)

In [3]:
# defining some concrete examples
monomer_examples = {
    'PVC' : MonomerExample(
        molname='polyvinylchloride',
        smiles='C=C-Cl',
        bond_map_nums=(1, 2),
        isotope_map={5 : 0, 6 : 0},
        assign_port_by=PortMethod.BOND_CLEAVAGE,
    ),
    'PE' : MonomerExample(
        molname = 'polyethylene',
        smiles = 'C=C',
        bond_map_nums=(1, 2),
        isotope_map = {3 : 0, 4 : 0},
        assign_port_by = PortMethod.BOND_CLEAVAGE
    ),
    'PAAm' : MonomerExample(
        molname = 'polyacrylamide',
        smiles = 'C=CC(=O)N',
        bond_map_nums=(1, 2),
        isotope_map = {7 : 0, 8 : 0},
        assign_port_by = PortMethod.BOND_CLEAVAGE
    ),
    'PS' : MonomerExample(
        molname = 'polystyrene',
        smiles = 'c1ccccc1C=C',
        bond_map_nums = (7, 8),
        isotope_map = {14 : 0, 16 : 0},
        assign_port_by = PortMethod.BOND_CLEAVAGE
    ),
    'Chitosan' : MonomerExample(
        molname = 'polyglucosamine',
        smiles = 'N[C@H]1C(O)OC(CO)[C@@H](O)[C@@H]1O',
        bond_map_nums=(7, 8),
        isotope_map = {17 : 0, 23 : 0},
        assign_port_by = PortMethod.MAP_NUMBERS
    )
}

# 1) Building monomer templates from basic SMILES

## Choose monomer HERE to define SMILES

In [ ]:
chosen_example = monomer_examples['PVC'] # feel at leisure to swap this out for another example
orig_smiles_mol = Chem.MolFromSmiles(chosen_example.smiles, sanitize=False)
display(orig_smiles_mol)

In [5]:
moldir = outdir / chosen_example.molname
moldir.mkdir(exist_ok=True, parents=True)

monodir = moldir / 'monomer_info'
monodir.mkdir(exist_ok=True)

## Expand SMILES to include full chemical info 
Namely, this includes explicit Hs, Kekulized aromatic rings, and atom map numbers

In [ ]:
from polymerist.polymers.monomers import specification
from pathlib import Path


new_smiles = specification.expanded_SMILES(chosen_example.smiles)
FULL_SMILES_MOL = Chem.MolFromSmiles(new_smiles, sanitize=False)
print(new_smiles)
display(FULL_SMILES_MOL)

## Assigning linkers for inter-monomer bonds

In [ ]:
from polymerist.rdutils.labeling import molwise
from polymerist.rdutils.bonding import portlib, dissolution


if portlib.get_num_ports(FULL_SMILES_MOL) == 0: # check for existence of ports to ensure idempotency
    if chosen_example.assign_port_by == PortMethod.MAP_NUMBERS:
        map_nums = chosen_example.isotope_map.keys()
        for (atom_id, map_num) in zip(molwise.atom_ids_by_map_nums(FULL_SMILES_MOL, *map_nums), map_nums):
            linker_atom = FULL_SMILES_MOL.GetAtomWithIdx(atom_id)
            linker_atom.SetIsotope(chosen_example.isotope_map[map_num])
            linker_atom.SetAtomicNum(0)

    elif chosen_example.assign_port_by == PortMethod.BOND_CLEAVAGE:
        rwmol = Chem.RWMol(FULL_SMILES_MOL)
        bond_atom_ids = molwise.atom_ids_by_map_nums(rwmol, *chosen_example.bond_map_nums)
        FULL_SMILES_MOL = dissolution.decrease_bond_order(rwmol, *bond_atom_ids)
        molwise.assign_ordered_atom_map_nums(FULL_SMILES_MOL, in_place=True)
        Chem.SanitizeMol(FULL_SMILES_MOL, sanitizeOps=specification.SANITIZE_AS_KEKULE) 

    elif chosen_example.assign_port_by == PortMethod.REACTION:
        raise NotImplemented
    else:
        raise TypeError(f'Must provide a valid port assignment method (cannot be of type {type(chosen_example.assign_port_by)})')

display(FULL_SMILES_MOL)

## Enumerating "cap" groups from linkers and generating spec-compliant SMARTS

In [ ]:
from copy import deepcopy
from polymerist.genutils.iteration import subsets


smarts = {}
sat_ids = subsets(portlib.get_linker_ids(FULL_SMILES_MOL), exclude_full=True)
for i, linkers_to_saturate in enumerate(sat_ids):
    new_mono = deepcopy(FULL_SMILES_MOL)
    for linker_id in linkers_to_saturate:
        linker_atom = new_mono.GetAtomWithIdx(linker_id)
        linker_atom.SetAtomicNum(1)
    molwise.assign_ordered_atom_map_nums(new_mono, in_place=True) # renumber with added atoms to preserve order correspondence
    Chem.SanitizeMol(new_mono, sanitizeOps=specification.SANITIZE_AS_KEKULE) # sanitize for good measure, making sure NOT to re-add aromaticity

    key = chosen_example.molname if not i else f'{chosen_example.molname}_TERM{i}'
    smarts[key] = [specification.compliant_mol_SMARTS(Chem.MolToSmiles(new_mono))] # for some reason, MolToSmarts doesn't reflect hydrogen addition

## Specify orientation of terminal monomers and create monomer representation object

In [ ]:
from polymerist.polymers.monomers import MonomerGroup
from polymerist.genutils.fileutils.pathutils import assemble_path

monogrp = MonomerGroup(
    monomers=smarts,
    term_orient={
        f'{chosen_example.molname}_TERM1' : 'head',
        f'{chosen_example.molname}_TERM2' : 'tail',
    }    
)

for (resname, rdmol) in monogrp.iter_rdmols():
    print(resname)
    display(rdmol)

mono_path = assemble_path(monodir, chosen_example.molname, extension='json')
monogrp.to_file(mono_path) # cache monomer SMARTS for future use

# 2) Generate coordinates and structure files with chemical info

In [10]:
pdbdir = moldir / 'pdb_structures'
pdbdir.mkdir(exist_ok=True)

sdfdir = moldir / 'sdf_structures'
sdfdir.mkdir(exist_ok=True)

## Grow chain of arbitrary DOP and generate PDB (only linear polymers currently supported)

In [ ]:
from polymerist.polymers.building import build_linear_polymer, mbmol_to_openmm_pdb
from polymerist.polymers import estimation

n_monomers : int = 30

chain = build_linear_polymer(monogrp, n_monomers=n_monomers, sequence='A', energy_minimize=True)
pdb_path = assemble_path(pdbdir, chosen_example.molname, extension='pdb')
mbmol_to_openmm_pdb(pdb_path, chain)
chain.visualize(backend='nglview')

## Fill in missing PDB chemical info with our OpenFF substructure match tools
Subsequently, partition into distinct residues and save to info-rich SDF format to avoid duplication of effort

In [ ]:
from openff.toolkit import Topology, Molecule
from polymerist.mdtools.openfftools import topology
from polymerist.mdtools.openfftools.partition import partition


# load PDB with substructure cover; if successful, refine into partition
offtop = Topology.from_pdb(pdb_path, _custom_substructures=monogrp.monomers)
was_partitioned = partition(offtop)
assert(was_partitioned)

# extract Molecule for Topology, set name and replace
offmol = topology.get_largest_offmol(offtop)
offmol.name = chosen_example.molname
offtop = offmol.to_topology()
display(offmol)

# serialize Topology to SDF to preserve fully-specified system
top_path = assemble_path(sdfdir, chosen_example.molname, extension='sdf')
topology.topology_to_sdf(top_path, offtop) 

## Repeat the above steps with atom cap to provide structure for RCT demo

In [ ]:
from polymerist.polymers import building, estimation

redux_n_monomers = estimation.estimate_n_monomers_infimum(monogrp, n_atoms_max=100)

redux_chain = building.build_linear_polymer(monogrp, n_monomers=redux_n_monomers, sequence='A', energy_minimize=True)
redux_pdb_path = assemble_path(pdbdir, chosen_example.molname, postfix='redux', extension='pdb')
building.mbmol_to_openmm_pdb(redux_pdb_path, redux_chain)
redux_chain.visualize(backend='nglview')

In [ ]:
redux_offtop = Topology.from_pdb(redux_pdb_path, _custom_substructures=monogrp.monomers)
was_partitioned = partition(redux_offtop)
assert(was_partitioned)

redux_offmol = topology.get_largest_offmol(redux_offtop)
redux_offmol.name = chosen_example.molname
redux_offtop = redux_offmol.to_topology()
# display(redux_offmol)

redux_top_path = assemble_path(sdfdir, chosen_example.molname, postfix='redux', extension='sdf')
topology.topology_to_sdf(redux_top_path, redux_offtop) # preserve fully-specified system in SDF format

# 3) Assignment of atomic partial charges

## Espaloma assignment (fast and direct)

In [ ]:
from polymerist.mdtools.openfftools.partialcharge import molchargers

charge_method = 'Espaloma-AM1-BCC'
charger = molchargers.MolCharger.subclass_registry[charge_method]()
espmol = charger.charge_molecule(offmol) # inherits name from base molecule

espmol_path = assemble_path(sdfdir, chosen_example.molname, postfix=charge_method, extension='sdf')
topology.topology_to_sdf(espmol_path, espmol.to_topology())

## RCT (reliable and provides charge templates)

### Perform explicit AM1-BCC-ELF10 (requires OpenEye on reduced-size polymer (THIS WILL TAKE A WHILE!)
Luckily, only needs to be done once per molecule

In [ ]:
charge_method = 'AM1-BCC-ELF10'
abe10_charger = molchargers.MolCharger.subclass_registry[charge_method]()
abe10mol = abe10_charger.charge_molecule(redux_offmol)

### Extract and save library charges, apply these to the target macromolecule (look how quick *this part* it is!)

In [ ]:
from polymerist.mdtools.openfftools.partialcharge.rescharge.calculation import compute_residue_charges
from polymerist.mdtools.openfftools.partialcharge.rescharge.interface import LibraryCharger


lib_chgs = compute_residue_charges(abe10mol, monogrp)
lib_chg_path = assemble_path(monodir, chosen_example.molname, postfix='library_charges', extension='json')
lib_chgs.to_file(lib_chg_path) # save library charges for future use

lib_charger = LibraryCharger(lib_chgs)
rctmol = lib_charger.charge_molecule(offmol)
rctmol_path = assemble_path(sdfdir, chosen_example.molname, postfix='RCT', extension='sdf')
topology.topology_to_sdf(rctmol_path, rctmol.to_topology())

# 4) Assign FF parameters and running simulations with OpenMM

In [22]:
ommdir = moldir / 'openmm'
ommdir.mkdir(exist_ok=True)

## Defining periodic box

In [23]:
import numpy as np
from openmm.unit import gram, centimeter, nanometer
from polymerist.mdtools.openfftools import boxvectors

top_path = espmol_path # choose path to desired parameterized & charged topology
# top_path = rctmol_path
offtop = topology.topology_from_sdf(top_path, allow_undefined_stereo=True)

exclusion = 1 * nanometer # how far beyond the tight bounding box of the polymer to extend the periodic box

# box_dims = np.array([4.0, 4.0, 4.0]) * nanometer
box_dims = boxvectors.get_topology_bbox(offtop)
box_vecs = boxvectors.box_vectors_flexible(box_dims)
box_vecs = boxvectors.pad_box_vectors_uniform(box_vecs, exclusion)
clear_output() # clear verbose stereochemistry warnings if present

## Packing box with solvent

In [29]:
from polymerist.mdtools.openfftools.solvation import solvents 
from polymerist.mdtools.openfftools.solvation.packing import pack_topology_with_solvent

solvent = solvents.water_TIP3P
rho = 0.997 * gram / centimeter**3

solv_top = pack_topology_with_solvent(offtop, solvent=solvent, box_vecs=box_vecs, density=rho, exclusion=exclusion)
solv_path = assemble_path(sdfdir, top_path.stem, postfix=f'solv_{solvent.name}', extension='sdf')
topology.topology_to_sdf(solv_path, solv_top)

2025-01-07 11:47:29.130 [INFO    :         packing:line 40  ] - Solvating 66.19464745668576 nm**3 Topology with 2207 water_TIP3P molecules to density of 0.997 g/(cm**3)
2025-01-07 11:47:32.024 [INFO    :         packing:line 49  ] - Packmol packing converged
2025-01-07 11:47:32.027 [INFO    :         packing:line 52  ] - Set solvated Topology box vectors to [[4.964100074768067 0.0 0.0] [0.0 3.5745000362396233 0.0] [0.0 0.0 3.7304999589920045]] nanometer


## Running simulations with OpenMM

### Defining reproducible and serializable simulation parameters

In [30]:
from openmm.unit import femtosecond, picosecond, nanosecond
from openmm.unit import kelvin, atmosphere
from polymerist.mdtools.openmmtools.parameters import (
    SimulationParameters,
    ThermoParameters,
    IntegratorParameters,
    ReporterParameters,
)
from polymerist.mdtools.openmmtools.reporters import DEFAULT_STATE_DATA_PROPS


# define reproducible simulation parameter set
sim_params = SimulationParameters(
    integ_params=IntegratorParameters(
        time_step=2*femtosecond, # 2 fs step doesn't work well for unconstrainted FFs
        total_time=2*nanosecond, # just a short simulation to demonstrate
        num_samples=50, # don't want to take too many samples
    ),
    thermo_params=ThermoParameters(
        ensemble='NVT',
        temperature=300*kelvin,
        friction_coeff=1*picosecond**-1, # required for Langevin Thermostat
    ),
    reporter_params=ReporterParameters(
        report_trajectory=True,
        traj_ext='dcd', # output to compressed binary trajectory files (recommended)
        report_state_data=True,
        state_data=DEFAULT_STATE_DATA_PROPS, # can tune these to taste
        report_checkpoint=True, # also keep checkpoints of OpenMM objects (specific to Context and machine)
        report_state=True,      # saving State is a bit redundant with checkpoints, but is machine-transferrable
    ),
)

# save simulation parameters to file (for reproducibility and idempotency)
sim_params_path = ommdir / 'sim_params.json'
sim_params.to_file(sim_params_path)

### Running a simulation "schedule" (serial sequence of simulations, each defined by their own SimulationParameters)

In [31]:
from polymerist.mdtools.openmmtools.parameters import SimulationParameters
from polymerist.mdtools.openmmtools.execution import run_simulation_schedule

from polymerist.mdtools.openfftools.omminter import openff_topology_to_openmm
from polymerist.mdtools.openfftools.topology import topology_from_sdf


sim_params = SimulationParameters.from_file(sim_params_path)
solv_top = topology_from_sdf(solv_path)


ff_name = 'openff-2.0.0'
schedule = {
    'demo_sim1' : sim_params
}

ommtop, ommsys, ommpos = openff_topology_to_openmm(solv_top, forcefield=ff_name, box_vecs=box_vecs)
history = run_simulation_schedule(ommdir, schedule, ommtop, ommsys, ommpos, return_history=True)

2025-01-07 11:50:43.296 [INFO    :      parameters:line 2894] - Attempting to up-convert vdW section from 0.3 to 0.4
2025-01-07 11:50:43.297 [INFO    :      parameters:line 2903] - Successfully up-converted vdW section from 0.3 to 0.4. `method="cutoff"` is now split into `periodic_method="cutoff"` and `nonperiodic_method="no-cutoff"`.
2025-01-07 11:50:43.308 [INFO    :      parameters:line 3094] - Attempting to up-convert Electrostatics section from 0.3 to 0.4
2025-01-07 11:50:43.308 [INFO    :      parameters:line 3104] - Successfully up-converted Electrostatics section from 0.3 to 0.4. `method="PME"` is now split into `periodic_potential="Ewald3D-ConductingBoundary"`, `nonperiodic_potential="Coulomb"`, and `exception_potential="Coulomb"`.
2025-01-07 11:50:50.592 [INFO    :       execution:line 38  ] - Initializing simulation 1/1 ("demo_sim1")
2025-01-07 11:50:50.595 [INFO    :          thermo:line 75  ] - Created LangevinMiddleIntegrator for NVT (Canonical) ensemble
2025-01-07 11:50: